#### Ensure gruyere files can be used

In [3]:
import sys
sys.path.append('.../inverse_design/gruyere/')

In [4]:
print(sys.path)

['/home/lucas/inverse_design/notebooks/gruyere_comparison', '/home/lucas/micromamba/envs/mamba_inverse_design/lib/python311.zip', '/home/lucas/micromamba/envs/mamba_inverse_design/lib/python3.11', '/home/lucas/micromamba/envs/mamba_inverse_design/lib/python3.11/lib-dynload', '', '/home/lucas/micromamba/envs/mamba_inverse_design/lib/python3.11/site-packages', '/home/lucas/micromamba/envs/mamba_inverse_design/lib/python3.11/site-packages/inverse_design-0.0.0-py3.11.egg', '../inverse_design/gruyere/', '.../inverse_design/gruyere/']


In [5]:
import kernex as kex
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

from gruyere.brushes import notched_square_brush, circular_brush, show_mask
from gruyere.design import _initialize_design, Design
from gruyere.generator import generate_design, _step_generator
from gruyere.states import DesignState

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 70

ModuleNotFoundError: No module named 'gruyere'

# We generate an initial random design 

In [ ]:
## We initalize a random array
# size = 30
size = 125
# size = 15

key = jax.random.PRNGKey(42)
latent_design = jax.random.uniform(key, (size,size)) * 2 - 1

In [ ]:
## We define the transform function
beta = 4
def transform(design, brush, beta=1):
    @jax.jit
    @kex.kmap(kernel_size=brush.shape, padding=('same','same'))
    def brush_conv(x):
        return jnp.sum(x * brush)

    return jnp.tanh(beta * brush_conv(design))

# Figure 2. (a)  diameter-13 circular brush

In [ ]:
circ_brush = circular_brush(9)
# circ_brush = circular_brush(3)
# circ_brush = circular_brush(13)
show_mask(circ_brush);

In [ ]:
transformed_design1 = transform(latent_design, circ_brush)

fig, ax = plt.subplots(1,2)
ax[0].imshow(latent_design);
ax[1].imshow(transformed_design1);

In [ ]:
circ_design = generate_design(transformed_design1, circ_brush)
# circ_design.show()

In [ ]:
plt.imshow(circ_design.x)

In [ ]:
from inverse_design.local_generator import generate_feasible_design_mask, dilate
from inverse_design.design import visualize

In [ ]:
# brush = circ_brush

# bg_mask = jnp.logical_or(bg_epsr>2, design_region)
# eroded = dilate(jnp.logical_not(bg_mask), brush)
# dilated = dilate(bg_epsr>2, brush)


# init_t_s = jnp.logical_not(jnp.logical_or(eroded, design_region)) 
# init_t_v = jnp.logical_not(jnp.logical_or(dilated, design_region))

In [ ]:
design = generate_feasible_design_mask(transformed_design1, circ_brush)
plt.imshow(design)

circ_design = generate_design(transformed_design1, circ_brush)
circ_design.show()

In [ ]:
fig, ax = plt.subplots(1,3)
ax[0].imshow(circ_design.x)
ax[1].imshow(design)
ax[2].imshow(design - circ_design.x)